# Vizualization of the results

In this notebook we just visualize and format the pickle file containing experiments results

In [ ]:
!ls -rtlh

In [ ]:
import pickle as pkl
import pandas as pd

pd.set_option("display.max_rows", 999)

filename = "benchmarks_2021-04-12-14:38:38.pickle"

with open(filename, "rb") as f:
    results = pkl.load(f)

print("commit number %s: datetime: %s" % (results["commit"], results["datetime"]))
    
df = results["results"]
df.head()

In [ ]:
metric = "avg_prec_w"

(
    df[["dataset", "classifier_title", metric, "repeat"]]
    .groupby(["dataset", "classifier_title"])
    # .agg(["mean", "std"])
    .agg(["mean"])
    .reset_index()
    .pivot(index="dataset", columns="classifier_title")
    .style
    .format("{:.2f}")
    .background_gradient(axis="index")    
)